In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle as pkl
import gc
import os

from src.simulation import sim_glm_pop
from src.theory import y_pred_full, covariance_full,  y_0_quad,  find_iso_rate, y_corrected_quad, find_iso_rate_input, cor_pred
from src.correlation_functions import rate, mean_by_region, tot_cross_covariance_matrix, two_pop_correlation, mean_pop_correlation, cov_to_cor
from src.plotting import raster_plot, abline
from src.generate_connectivity import excitatory_only, gen_adjacency, hippo_weights, macro_weights




In [26]:
def kth_term(J,R, k):
    result = np.zeros_like(J)
    for m in range(k):
        n = k -m 
        result +=  np.linalg.matrix_power(J, m) @ R @  np.linalg.matrix_power(J, n).T
    return result

def cov_terms(J0, g, h, b, N):
    J = macro_weights(J = J0, h3 = h, h1 = h, g = g)
    r = y_0_quad(J, b)
    R = np.diag(r/N)
    gain =  2*(J@r+b)
    J_lin =J* gain[...,None]
    t1 = kth_term(J_lin, R, 1)[3,3]
    t2 = kth_term(J_lin, R, 2)[3,3]
    t3 = kth_term(J_lin, R, 3)[3,3]
    return (t1, t2, t3)


In [27]:
J0 = .2
g = 3
h = 2
b = np.array([.5, .5, .7, .5, .5, .7])
N_E =60
N_I = 15
Ns =np.array([N_E, N_E, N_I,  N_E, N_E, N_I])

cov_terms(J0 = J0, g = g, h = h, b = b, N = Ns)

(0.0, 0.0045465140586380855, -0.007036833494094149)

In [28]:
cov_terms2(J0 = J0, g = g, h = h, b = b, N = Ns)

(0.0, 0.0047694965066064675, -0.007204599029977917)